In [ ]:
#@title 🏥 Pharmacy Start Address

#!pip install openrouteservice  -q
# https://account.heigit.org/manage/key
# https://openrouteservice-py.readthedocs.io/en/latest/

import ipywidgets as widgets
from IPython.display import display, clear_output

import requests
import time

try:
  import openrouteservice
  from openrouteservice.optimization import Vehicle, Job
  from openrouteservice import Client
except:
  !pip install openrouteservice  -q
  import openrouteservice
  from openrouteservice.optimization import Vehicle, Job


import urllib.parse

#import folium
#from folium import plugins
from IPython.display import display, HTML
# ── requirements ────────────────────────────────────────────────
# pip install openrouteservice scikit-learn python-dotenv

from sklearn.cluster import KMeans
import datetime as dt
import urllib.parse
import requests
import time
from typing import Dict, Tuple


api_key = "5b3ce3597851110001cf624872f25c4e63de429880fd7c7e9e1727b2"

# --- Cell 1: Control Panel ---

description_style = {'description_width': "initial"}
field_width = '400px'


# PHARMACY OPTIONS
pharmacy_options = {
    "University Pl Pharmacy": "23 University Pl Blvd, Jersey City, NJ 07305",
    "MediSave Pharmacy": "362 Monroe St , Passaic, NJ 07055",
    "Other (enter manually below)": ""
}

pharmacy_dropdown = widgets.Dropdown(
    options=list(pharmacy_options.keys()),
    value="University Pl Pharmacy",
    description="🏥 Pharmacy:",
    layout=widgets.Layout(width='50%'),
    style=description_style

)

pharmacy_manual_input = widgets.Text(
    placeholder="Enter custom pharmacy address...",
    description="✍️ Custom Starting address:",
    layout=widgets.Layout(width='50%'),
    style=description_style

)

# NUMBER OF ADDRESSES
num_fields_input = widgets.BoundedIntText(
    value=20,
    min=1,
    max=50,
    step=1,
    description="🧾 # Addresses:",
    layout=widgets.Layout(width='250px'),
    style=description_style
)

# NUMBER OF DRIVERS
num_drivers_input = widgets.BoundedIntText(
    value=2,
    min=1,
    max=10,
    step=1,
    description="🚚 # Drivers:",
    layout=widgets.Layout(width='250px'),
    style=description_style
)




time_now_input = widgets.Text(
    placeholder="10:00",
    description="✍️ time_now_input:",
    layout=widgets.Layout(width='250px'),
    style=description_style

)

end_of_day = widgets.Text(
    placeholder="18:00",
    description="✍️ end_of_day:",
    layout=widgets.Layout(width='250px'),
    style=description_style
)

Facility_delivery_time = widgets.Text(
    placeholder="13:30",
    description="✍️ Facility_delivery_time:",
    layout=widgets.Layout(width='250px'),
    style=description_style
)

# DISPLAY MAIN CONFIG
# Display vertically with spacing
display(
    widgets.VBox([
        pharmacy_dropdown,
        pharmacy_manual_input,
        num_fields_input,
        num_drivers_input,
        time_now_input,
        end_of_day,
        Facility_delivery_time
    ], layout=widgets.Layout(gap='10px'))
)



#print("✅✅✅✅✅✅✅")


#@title 📦 Delivery Addresses & Driver Constraints

# --- Output Areas ---
addresses_output = widgets.Output()
drivers_output = widgets.Output()
display(addresses_output, drivers_output, )

# --- Storage ---
address_widgets = []
priority_widgets = []
driver_widgets = []
urgency_widgets = []
refrigerated_widgets = []
time_window_widgets = []

# --- Helper ---
def get_pharmacy_address():
    selected = pharmacy_dropdown.value
    return pharmacy_manual_input.value.strip() if selected == "Other (enter manually below)" else pharmacy_options[selected]

# --- Create Address Fields ---
# --- Create Address Fields ---
def create_address_fields(num_fields):
    address_widgets.clear()
    priority_widgets.clear()
    urgency_widgets.clear()
    refrigerated_widgets.clear()
    time_window_widgets.clear()
    addresses_output.clear_output(wait=True)

    with addresses_output:
        print(f"📦 Delivery Addresses (Start at: {get_pharmacy_address()}):")
        rows = []
        for i in range(num_fields):
            addr = widgets.Text(
                placeholder="Enter address",
                description=f"Address {i+1}:",
                layout=widgets.Layout(width='35%')
            )

            prio = widgets.Dropdown(
                options=[str(p) for p in range(1, 10)],
                value='5',
                description="Priority",
                layout=widgets.Layout(width='130px')
            )

            urgent = widgets.Checkbox(
                value=False,
                description="Urgent",
                indent=False,
                layout=widgets.Layout(width='100px')
            )

            refrigerated = widgets.Checkbox(
                value=False,
                description="Refrigerated",
                indent=False,
                layout=widgets.Layout(width='100px')
            )

            time_window = widgets.Text(
                placeholder="e.g., 14:00-16:30",
                description="Time",
                layout=widgets.Layout(width='180px')
            )

            address_widgets.append(addr)
            priority_widgets.append(prio)
            urgency_widgets.append(urgent)
            refrigerated_widgets.append(refrigerated)

            time_window_widgets.append(time_window)

            rows.append(widgets.HBox([addr, prio, urgent, time_window, refrigerated]))

        # Batch display in chunks (e.g., 5 at a time)
        batch_size = 5
        display(widgets.VBox([widgets.VBox(rows[i:i + batch_size]) for i in range(0, len(rows), batch_size)]))

# --- Create Driver Fields ---
def create_driver_fields(num_drivers, default_max_tasks):
    driver_widgets.clear()
    drivers_output.clear_output(wait=True)

    with drivers_output:
        print("🚚 Driver Constraints:")
        rows = []
        for i in range(num_drivers):
            name = widgets.Text(
                value=f"Driver{i+1}",
                description="Name",
                layout=widgets.Layout(width='200px')
            )
            max_tasks = widgets.BoundedIntText(
                value=default_max_tasks,
                min=1,
                max=100,
                description="Max Tasks",
                layout=widgets.Layout(width='150px')
            )
            roundtrip = widgets.Checkbox(value=False, description="Round Trip")
            home = widgets.Text(
                placeholder="End address (if not round trip)",
                description="Driver's Home Address",
                layout=widgets.Layout(width='30%'),
                style={'description_width': '100px'}
            )
            driver_widgets.append((name, max_tasks, roundtrip, home))
            rows.append(widgets.HBox([name, max_tasks, roundtrip, home]))

        display(widgets.VBox(rows) )

# --- Callback ---
def refresh_fields(*args):
    create_address_fields(num_fields_input.value)
    create_driver_fields(num_drivers_input.value, num_fields_input.value)

# --- Observe Triggers (run only once) ---
num_fields_input.unobserve_all()
num_drivers_input.unobserve_all()
#pharmacy_dropdown.unobserve_all()
#pharmacy_manual_input.unobserve_all()

num_fields_input.observe(refresh_fields, names='value')
num_drivers_input.observe(refresh_fields, names='value')
pharmacy_dropdown.observe(refresh_fields, names='value')
pharmacy_manual_input.observe(refresh_fields, names='value')

# --- Initial Render ---
refresh_fields()



# Global cache: address -> (lon, lat)
_geocode_cache: Dict[str, Tuple[float, float]] = {}

In [ ]:
#@title 📦 Results


# ── helper ──────────────────────────────────────────────────────
def _sec_today(hh_mm: str) -> int:
    """convert 'HH:MM' to seconds since midnight"""
    h, m = map(int, hh_mm.split(":"))
    return h * 3600 + m * 60

def geocode_addr(client: Client, address: str):  # geocode_addr
    """lightweight wrapper around ORS Pelias search -> [lon, lat]"""
    if address in _geocode_cache:
        return _geocode_cache[address]
    rsp = client.pelias_search(text=address)
    if not rsp["features"]:
        raise ValueError(f"Could not geocode: {address}")
    coords = rsp["features"][0]["geometry"]["coordinates"]
    _geocode_cache[address] = coords  # store in cache
    return coords

def geocode_free(ors, address: str) -> Tuple[float, float]: # geocode_free
    """Geocode using a dict cache + maps.co fallback."""
    if address in _geocode_cache:
        return _geocode_cache[address]
    time.sleep(1)
    url = "https://geocode.maps.co/search"
    params = {"q": address, "api_key": "6861820819cc3625503686uwk18b7ea"}
    resp = requests.get(url, params=params)
    resp.raise_for_status()
    data = resp.json()
    if not data:
        raise ValueError(f"Could not geocode: {address}")
    coords = (float(data[0]["lon"]), float(data[0]["lat"]))
    _geocode_cache[address] = coords
    return coords

def hhmm_from_sec(sec: int) -> str:
    sec %= 24 * 3600
    return f"{sec//3600:02d}:{sec%3600//60:02d}"

def reverse_geocode_addr(client: Client, lon: float, lat: float) -> str:
    rsp = client.pelias_reverse(point=[lon, lat], size=1)
    feats = rsp.get("features", [])
    return feats[0]["properties"]["label"] if feats else "unknown"

def optimize_pharmacy_routes(
    api_key: str,
    pharmacy_address: str,
    patient_addresses: list[str],
    urgency_flags: list[bool],
    driver_settings: list[dict],
    now: str = "10:00",
    end_day: str = "18:00",
    urgent_due: str = "13:30",
):
    if len(patient_addresses) != len(urgency_flags):
        raise ValueError("patient_addresses and urgency_flags must align")
    n_drivers = len(driver_settings)
    now_s = _sec_today(now)
    end_day_s = _sec_today(end_day)
    urgent_due_s = _sec_today(urgent_due)
    ors = Client(key=api_key, timeout=180)
    _geocode_cache[pharmacy_address] = geocode_addr(ors, pharmacy_address)
    home_addresses = [d["home"] for d in driver_settings if d["home"]]
    for addr in patient_addresses + home_addresses:
        _geocode_cache.setdefault(addr, geocode_addr(ors, addr))
    pts = [_geocode_cache[a] for a in patient_addresses]
    kmeans = KMeans(n_clusters=n_drivers, n_init="auto").fit(pts)
    cluster_ids = kmeans.labels_
    jobs = []
    for j, (addr, urgent) in enumerate(zip(patient_addresses, urgency_flags)):
        cid = int(cluster_ids[j])
        tw = [now_s, urgent_due_s] if urgent else [now_s, end_day_s]
        jobs.append(
            Job(
                id=j,
                location=_geocode_cache[addr],
                skills=[cid],
                priority=10 if urgent else 1,
                service=900,
                amount=[1],
                time_windows=[tw],
            )
        )
    vehicles = []
    for i, drivr in enumerate(driver_settings, start=1):
        vid = i - 1
        if drivr["round_trip"]:
            vehicles.append(
                Vehicle(
                    id=i,
                    profile="driving-car",
                    start=_geocode_cache[pharmacy_address],
                    end=_geocode_cache[pharmacy_address],
                    skills=[vid],
                    capacity=[drivr["max_tasks"]],
                    time_window=[now_s, end_day_s],
                )
            )
        else:
            end_loc = _geocode_cache[drivr["home"]] if drivr["home"] else None
            vehicles.append(
                Vehicle(
                    id=i,
                    profile="driving-car",
                    start=_geocode_cache[pharmacy_address],
                    end=end_loc,
                    skills=[vid],
                    capacity=[drivr["max_tasks"]],
                    time_window=[now_s, end_day_s],
                )
            )
    result = ors.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
    return result

pharmacy_address = get_pharmacy_address()
patient_addresses = [w.value.strip() for w in address_widgets if w.value.strip()]
urgency_flags = [w.value for w in urgency_widgets][: len(patient_addresses)]

driver_settings = []
for name, max_tasks, round_trip, home in driver_widgets:
    driver_settings.append({
        "name": name.value.strip(),
        "max_tasks": int(max_tasks.value),
        "round_trip": round_trip.value,
        "home": home.value.strip(),
    })

result = optimize_pharmacy_routes(
    api_key,
    pharmacy_address,
    patient_addresses,
    urgency_flags,
    driver_settings,
    now=time_now_input.value or "10:00",
    end_day=end_of_day.value or "18:00",
    urgent_due=Facility_delivery_time.value or "13:30",
)

print("\n=== DAILY PLAN ===")
all_routes = []

for route in result["routes"]:
    vehicle_id = route["vehicle"]
    driver_name = driver_settings[vehicle_id - 1]["name"]

    dur = route["duration"] / 30
    dist = route["distance"] / 1609.34
    stops = [step["id"] for step in route["steps"] if step["type"] == "job"]
    print(f"\n 🚚  Driver {vehicle_id}: - {driver_name} 👦 {len(stops)} stops | {dur:.1f} min | {dist:.2f} miles")

    steps = route['steps']
    ordered_addresses = [pharmacy_address]
    print(f"  🏁{pharmacy_address}")
    for step in steps:
        if step["type"] == "job":
            job_id = step["job"]
            addr = patient_addresses[job_id]
            time_of_arrival = hhmm_from_sec(step["arrival"])
            tag = "URGENT" if urgency_flags[job_id] else ""
            print(f"  📍{addr}  ⏰  {time_of_arrival} {tag}")
            ordered_addresses.append(addr)
        elif step["type"] == "end":
            end_station = driver_settings[vehicle_id - 1]["home"] or pharmacy_address
            ordered_addresses.append(end_station)
            print(f"  🏁{end_station}")
            #print("====end of trip=====")
    encoded = [urllib.parse.quote_plus(a) for a in ordered_addresses]
    maps_url = "https://www.google.com/maps/dir/" + "/".join(encoded)
    total_time = round(route['duration'] / 30, 1)
    print("🔗 Google Maps:", maps_url)
    print("-" * 60)

    all_routes.append({
        "vehicle_id": vehicle_id,
        "driver_name": driver_name,
        "addresses": ordered_addresses,
        "google_maps_link": maps_url,
        "total_minutes": total_time,
        "steps":steps
    })

#for route in all_routes:
#    print(f"\n🚚 Driver #{route['vehicle_id']} - {route['driver_name']}")
#    for st, stop in enumerate(route['addresses']):
#        print("  📍", stop,"⏰", hhmm_from_sec(route['steps'][st]["arrival"]))
#
#    print(f"⏱️ Estimated Time: {route['total_minutes']} minutes")
